# An Adventure Game: Dungeon Dice Monsters

We're going to write the basics of a text-based adventure game.

First we're going to "steal" our Die class from the OOP notebook. **Note**: this could be put in a different file and imported.

In [1]:
import random
import die

test_die = die.Die()
test_die()

3

We will define a class `Creature` that will be the parent of `Player`s and `Monster`s.

In [2]:
DEF_HEALTH = 12
DEF_STRENGTH = die.DEF_FACES


class Participant(object):
    '''This class will be the parent of Player and Monster.
       Participants will have a name, a location (?),
       health rating, and strength rating.
    '''
    def __init__(self, name, health=DEF_HEALTH, strength=DEF_STRENGTH):
        '''Initialize a Creature.'''
        self.name = name
        self.health = health
        self.strength = strength
        self.alive = True
        self.dungeon = None
        self.visible = True
        self.can_disappear = False
        
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return "I am {} with health of {} and strength of {}".format(self.name,
                                                                     self.health,
                                                                     self.strength)
    
    def hide(self):
        '''Make this creature hide in the shadows.'''
        if self.can_disappear:
            self.visible = False

    def show(self):
        '''Make this monster come out of the shadows.'''
        self.visible = True

    def is_visible(self):
        '''Is this creature visible?'''
        return self.visible

    def can_reveal(self):
        return False

    def take_a_hit(self, damage):
        '''Suffer a blow from a foe.'''
        self.health -= damage
        if self.health <= 0:
            self.alive = False

    # also add super_reveal()!
    def reveal(self):
        if self.dungeon is None:
            print("You are a real nowhere creature!")
            return []
        else:
            return self.dungeon.get_monsters(self)
            
    def locate(self, dungeon):
        self.dungeon = dungeon

In [3]:
cr1 = Participant("John Li")
print(cr1)
cr1.take_a_hit(4)
cr1.take_a_hit(4)
print(repr(cr1))
cr1.reveal()

John Li
I am John Li with health of 4 and strength of 6
You are a real nowhere creature!


[]

Let's set up the descendant classes of `Participant`:

In [4]:
class Monster(Participant):
    '''A dangerous, dice-rolling monster!'''
    def __init__(self, name, health=DEF_HEALTH, strength=DEF_STRENGTH):
        super().__init__(name, health=health, strength=strength)
        self.die = die.Die(strength)
        self.visible = True
    
    def attack(self):
        '''Attack a foe with a certain force.'''
        return self.die()
    
class VanishingMonster(Monster):
    '''A dangerous monster that can disappear!'''
    def __init__(self, name, health=DEF_HEALTH, strength=DEF_STRENGTH):
        super().__init__(name, health=health, strength=strength)
        self.die = die.Die(strength)
        self.visible = True
        self.can_disappear = True

In [5]:
blob = Monster("The Blob")
print("Blob attack:", blob.attack())
blob.take_a_hit(2)
print(blob)
print(repr(blob))
dark_magic = Monster("Dark Magician")
blue_eyes = Monster("Endless Dragon with Blue Eyes")
yz_dragon = VanishingMonster("YZ-track Dragon")
yz_dragon.hide()

Blob attack: 3
The Blob
I am The Blob with health of 10 and strength of 6


In [6]:
ATTACK_RATIO = 2
MAGE_ATTACK_DOWN = 1
HEALER_ATTACK_DOWN = 2


class Player(Participant):
    '''A heroic adventurer facing dice monsters in a dungeon.'''
    def __init__(self, name):
        super().__init__(name)
        
    def attack(self):
        '''Attack a foe with a certain force.'''
        return self.strength // ATTACK_RATIO
    
class Mage(Player):
    '''A mage: can reveal.'''
    def __init__(self, name):
        super().__init__(name)
        
    def attack(self):
        '''Attack a foe with a certain force.'''
        return super().attack() - MAGE_ATTACK_DOWN

    def can_reveal(self):
        return True

        
class Healer(Player):
    '''A healer: can increase health of teammates.'''
    def __init__(self, name):
        super().__init__(name)
        
            
    def attack(self):
        '''Attack a foe with a certain force.'''
        return super().attack() - HEALER_ATTACK_DOWN

In [7]:
michael = Player("Michael Lippe")
print(repr(michael))
print("Michael attack:", michael.attack())
pablo = Player("Pablo Parra")
somya = Mage("Somya Mehta")
yaseen = Player("Yaseen Elsebaie")
hailey = Player("Hailey Lint")

I am Michael Lippe with health of 12 and strength of 6
Michael attack: 3


In [8]:
DEF_HEIGHT = 4
DEF_WIDTH = 4


class Dungeon(object):
    '''A place for monsters and players to roam and battle.'''
    def __init__(self, name, width=DEF_WIDTH, height=DEF_HEIGHT):
        self.name = name
        self.width = width
        self.height = height
        self.participants = {}  # dict for now!
        self.part_locs = {}

    def __getitem__(self, name):
        return self.participants[name]

    def __str__(self):
        return self.name

    def __repr__(self):
        inhabs = ["\t{}\n".format(name)  # expr
                  for (name, particip) in self.participants.items()  # for
                  if particip.is_visible()  # cond
                 ]
        inhabs_str = ''.join(inhabs)
        return str(self) + "\nInhabitants:\n" + inhabs_str
    
    def get_particip_of_type(self, particip_type):
        '''Get all participures of `type` in this dungeon.'''
        return [particip  # expr
                for particip in self.participants.values()  # for loop
                if isinstance(particip, particip_type) and particip.is_visible()]  # cond
        
    def get_monsters(self, particip):
        '''Get all monsters in this dungeon.'''
        if particip.can_reveal():
            return self.get_particip_of_type(Monster)
        else:
            print("You have no power to reveal what lurks in the darkness!")
            return []
    
    def get_players(self):
        '''Get all players in this dungeon.'''
        return self.get_particip_of_type(Player)
    
    def get_rand_x(self):
        return random.randint(0, self.width - 1)
        
    def get_rand_y(self):
        return random.randint(0, self.height - 1)

    def is_full(self):
        '''Is there any room on the grid?'''
        return len(self.part_locs) >= self.width * self.height

    def get_free_loc(self):
        '''Find a spot that is free.'''
        x = self.get_rand_x()
        y = self.get_rand_y()
        loc = (x, y)
        # Check if this spot is occupied:
        while loc in self.part_locs:
            print("Looking for a free spot!")
            x = self.get_rand_x()
            y = self.get_rand_y()
            loc = (x, y)
        return loc
        
    def add_partcip(self, particip):
        '''Add a participure to this object.
        Should we allow re-adding?'''
        if not self.is_full():
            self.participants[str(particip)] = particip
            free_loc = self.get_free_loc()
            self.part_locs[free_loc] = particip
            particip.locate(self)
        else:
            print("Sorry, no room in {}; try somewhere else!".format(str(self)))
    
    def meet(self, particip1, particip2):
        '''Two denizens of the dungeon confront each other.'''
        if str(particip1) not in self.participants:
            print("{} is not in the {}".format(str(particip1), str(self)))
        elif str(particip2) not in self.participants:
            print("{} is not in the {}".format(str(particip2), str(self)))
        elif isinstance(particip1, Player) and isinstance(particip2, Player):
            print("Hi {} I am {}".format(str(particip1), str(particip2)))
        elif isinstance(particip1, Monster) and isinstance(particip2, Monster):
            print("Aaargh! Growl!")
        else:
            print("{} is fighting {}".format(str(particip1), str(particip2)))
            particip1.take_a_hit(particip2.attack())
            particip2.take_a_hit(particip1.attack())

In [9]:
catacomb = Dungeon("Catacomb of Horrors")
catacomb.add_partcip(blob)
catacomb.add_partcip(michael)
catacomb.add_partcip(somya)
catacomb.add_partcip(dark_magic)
catacomb.add_partcip(pablo)
catacomb.add_partcip(hailey)
catacomb.add_partcip(blue_eyes)
catacomb.add_partcip(yz_dragon)
print("String version:", catacomb)
print(repr(catacomb))
catacomb["The Blob"]
# help(catacomb.creatures)
# print(catacomb.creat_locs)

Looking for a free spot!
String version: Catacomb of Horrors
Catacomb of Horrors
Inhabitants:
	The Blob
	Michael Lippe
	Somya Mehta
	Dark Magician
	Pablo Parra
	Hailey Lint
	Endless Dragon with Blue Eyes



I am The Blob with health of 10 and strength of 6

In [21]:
print([str(monster) for monster in michael.reveal()])
# I don't want long-ass descriptions like print will give me!
# print(somya.reveal())
# So instead:
print([str(monster) for monster in somya.reveal() if "Blob" in str(monster)])
# the above as a loop:
monster_names = []
for monster in somya.reveal():
    if "Blob" in str(monster):
        monster_names.append(str(monster))
print(monster_names)

You have no power to reveal what lurks in the darkness!
[]
['The Blob']
['The Blob']


In [11]:
catacomb.meet(blob, michael)
# debug prints:
# print(repr(blob))
# print(repr(michael))

The Blob is fighting Michael Lippe


In [12]:
catacomb.meet(pablo, michael)

Hi Pablo Parra I am Michael Lippe


In [13]:
catacomb.meet(blob, yaseen)

Yaseen Elsebaie is not in the Catacomb of Horrors
